<a href="https://colab.research.google.com/github/DSscherbinin/practical_ml/blob/main/Convolutional_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T
from tqdm.auto import tqdm
import seaborn as sns
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from torchvision.datasets import MNIST

In [ ]:
from torch.utils.data import DataLoader
transform = T.Compose(
    [T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,))]
)

train_dataset = MNIST('MNIST', download=True, train=True, transform=transform)
val_dataset = MNIST('MNIST', download=True, train=False, transform=transform)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, num_workers=4)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1024, num_workers=4)


In [ ]:
def train_epoch(model, optimizer, criterion, train_loader, device=device):
    loss_log, acc_log = [], []
    model.train()
    for batch_num, (x_batch, y_batch) in enumerate(train_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        logits = model(x_batch).to(device)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        loss_log.append(loss.item())
        pred = torch.max(logits, 1)[1].data.cpu().numpy()
        acc = np.mean(pred == y_batch.data.cpu().numpy())
        acc_log.append(acc)
    return loss_log, acc_log

def test(model, criterion, val_loader, device=device):
    loss_log, acc_log = [], []
    model.eval()
    for batch_num, (x_batch, y_batch) in enumerate(val_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        with torch.no_grad():
            logits = model(x_batch).to(device)
            loss = criterion(logits, y_batch)
        pred = torch.max(logits, 1)[1].data.cpu().numpy()
        acc = np.mean(pred == y_batch.data.cpu().numpy())
        acc_log.append(acc)
        loss = loss.item()
        loss_log.append(loss)
    return loss_log, acc_log

def train(model, optimizer, criterion, n_epochs, batch_size, train_loader, val_loader, device=device):
    train_log, train_acc_log = [], []
    val_log, val_acc_log = [], []
    for epoch in tqdm(range(n_epochs)):
        train_loss, train_acc = train_epoch(model, optimizer, criterion, train_loader, device=device)
        val_loss, val_acc = test(model, criterion, val_loader)
        train_log.extend(train_loss)
        train_acc_log.extend(train_acc)
        steps = train_dataset.train_labels.shape[0] / batch_size
        val_log.append((steps * (epoch + 1), np.mean(val_loss)))
        val_acc_log.append((steps * (epoch + 1), np.mean(val_acc)))
    return {'train_logloss': np.mean(train_loss), 'train_accuracy': np.mean(train_acc),
            'val_logloss': val_log[-1], 'val_accuracy': val_acc_log[-1]}

class MyConNet(nn.Module):
    def __init__(self, image_channels=1):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=image_channels, out_channels=6, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=5)
        )
        self.head = nn.Sequential(
            nn.Linear(in_features=90, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10)
        )

    def forward(self, x):
        out = self.encoder(x)
        out = nn.Flatten()(out)
        out = self.head(out)
        return out

    def get_embedding(self, x):
        out = self.encoder(x)
        return out


EPOCHS = 10
model = MyConNet().to(device)
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters())
history = train(model, opt, criterion, EPOCHS, batch_size=32, train_loader=train_loader, val_loader=val_loader, device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [ ]:
history

{'train_logloss': 0.017964200203402895,
 'train_accuracy': 0.9941166666666666,
 'val_logloss': (18750.0, 0.048581971786916256),
 'val_accuracy': (18750.0, 0.9875538105867347)}